Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

Importing the training data

In [2]:
mydata = pd.read_csv('../input/titanic/train.csv')
mydata.info()
mydata.isnull().sum() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Taking care of missing data for age**

In [3]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(mydata[['Age']])
mydata[['Age']] = imputer.transform(mydata[['Age']])

**Taking care of missing data for embarked city & Creating dummy variables for embarked city**

In [4]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(mydata[['Embarked']])
mydata[['Embarked']] = imputer.transform(mydata[['Embarked']])

emb=pd.get_dummies(mydata['Embarked'])
mydata = pd.concat([mydata, emb], axis=1, join='inner')

**Encoding Sex**

In [5]:
sex=pd.get_dummies(mydata['Sex'])
mydata=pd.concat([mydata, sex], axis=1, join='inner')
mydata.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S,female,male
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,0,1,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0,1,0,1


**Creatin an additional parameter for traveling alone**

In [6]:
alone = []
par=mydata['Parch']; sib=mydata['SibSp']
for i in range(0, len(mydata['PassengerId'])):
    if (par[i]+sib[i]==0)==True:
        alone.append(1)
    else:
        alone.append(0)
alone= pd.DataFrame(alone, columns =['Alone'])
mydata = pd.concat([mydata, alone], axis=1, join='inner')

**Making survival the last column in the dataframe**

In [7]:
surv= mydata['Survived']
del mydata['Survived']
surv = pd.DataFrame(surv, columns =['Survived'])
mydata = pd.concat([mydata, surv], axis=1, join='inner')
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     891 non-null    object 
 11  C            891 non-null    uint8  
 12  Q            891 non-null    uint8  
 13  S            891 non-null    uint8  
 14  female       891 non-null    uint8  
 15  male         891 non-null    uint8  
 16  Alone        891 non-null    int64  
 17  Survived     891 non-null    int64  
dtypes: float64(2), int64(6), object(5), uint8(5)
memor

**Uploading the test data**

In [8]:
tdata=pd.read_csv('../input/titanic/test.csv')
tdata.info()
tdata.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

**Taking care of missing data for age & fare**

In [9]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(tdata[['Age']])
tdata[['Age']] = imputer.transform(tdata[['Age']])
imputer.fit(tdata[['Fare']])
tdata[['Fare']] = imputer.transform(tdata[['Fare']])

**Creating dummy variables for embarked city**

In [10]:
emb=pd.get_dummies(tdata['Embarked'])
tdata=pd.concat([tdata, emb], axis=1, join='inner')

**Encoding Sex**

In [11]:
sex=pd.get_dummies(tdata['Sex'])
tdata=pd.concat([tdata, sex], axis=1, join='inner')

tdata.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C,Q,S,female,male
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1,0,0,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,0,1,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1,0,0,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0,1,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,0,1,1,0


**Creating an additional parameter for traveling alone**

In [12]:
alone = []
par=tdata['Parch']; sib=tdata['SibSp']
for i in range(0, len(tdata['PassengerId'])):
    if (par[i]+sib[i]==0)==True:
        alone.append(1)
    else:
        alone.append(0)
alone= pd.DataFrame(alone, columns =['Alone'])
tdata = pd.concat([tdata, alone], axis=1, join='inner')

**Removing the columns that won't be used**

In [13]:
pass_id=tdata['PassengerId']
mydata=mydata.drop(['PassengerId','Name', 'Sex', 'Embarked', 'Ticket', 'Cabin'], axis = 1)
tdata=tdata.drop(['PassengerId','Name','Sex','Embarked', 'Ticket', 'Cabin'], axis = 1)
mydata.info()
tdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   C         891 non-null    uint8  
 6   Q         891 non-null    uint8  
 7   S         891 non-null    uint8  
 8   female    891 non-null    uint8  
 9   male      891 non-null    uint8  
 10  Alone     891 non-null    int64  
 11  Survived  891 non-null    int64  
dtypes: float64(2), int64(5), uint8(5)
memory usage: 53.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Age     418 non-null    float64
 2   SibSp   418 non-null    int64 

**Creating the arrays for training data & Feature scaling**

In [14]:
#prepare the x and y training data
X_train = mydata.iloc[:, :-1].values
y_train = mydata.iloc[:, -1].values

#feature scaling 
sc = StandardScaler()
X_train[:, [1,2,3,4]] = sc.fit_transform(X_train[:, [1,2,3,4]])

#prepare x_test
X_test=tdata.iloc[:, :].values

#feature scaling 
sc = StandardScaler()
X_test[:, [1,2,3,4]] = sc.fit_transform(X_test[:, [1,2,3,4]])

**Training the random forest model**

In [15]:
classifier = RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=1)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=1)

**Predicting the results**

In [16]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1))))

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0
 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 0 1 0 1 1 0 1 0 0 1]


In [17]:
final_surv = pd.DataFrame(y_pred, columns = ['Survived'])
final = pd.concat([pass_id, final_surv], axis=1, join='inner')
final.to_csv(r'./Submission.csv', index=False)